# Script for a period

In [1]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [2]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [3]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [4]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [5]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, project_name):
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return all_campaign_data

# Fetch campaign data for each project
all_campaign_data_guten = fetch_campaign_data(campaign_chunks_guten, headers_guten, 'WB-GutenTech')
all_campaign_data_giper = fetch_campaign_data(campaign_chunks_giper, headers_giper, 'WB-ГиперМаркет')
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks_kitchen, headers_kitchen, 'WB-KitchenAid')
all_campaign_data_smart = fetch_campaign_data(campaign_chunks_smart, headers_smart, 'WB-Smart-Market')

# Create DataFrames from the campaign data
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

# Sort each DataFrame by 'createTime' in descending order
campaigns_guten = campaigns_guten.sort_values(by='createTime', ascending=False)
campaigns_giper = campaigns_giper.sort_values(by='createTime', ascending=False)
campaigns_kitchen = campaigns_kitchen.sort_values(by='createTime', ascending=False)
campaigns_smart = campaigns_smart.sort_values(by='createTime', ascending=False)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully for WB-GutenTech chunk 1
Data retrieved successfully for WB-GutenTech chunk 2
Data retrieved successfully for WB-GutenTech chunk 3
Data retrieved successfully for WB-GutenTech chunk 4
Data retrieved successfully for WB-GutenTech chunk 5
Data retrieved successfully for WB-GutenTech chunk 6
Data retrieved successfully for WB-GutenTech chunk 7
Data retrieved successfully for WB-GutenTech chunk 8
Data retrieved successfully for WB-GutenTech chunk 9
Data retrieved successfully for WB-ГиперМаркет chunk 1
Data retrieved successfully for WB-KitchenAid chunk 1
Data retrieved successfully for WB-Smart-Market chunk 1
Combined Campaign Data


In [6]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-02-19,2025-02-19,Mr Proper_19.02.2025,23408417,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-02-12,2025-03-14,Venus_12.02.2025,23231292,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-02-12,2025-03-10,Always_12.02.2025,23230882,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры на запястье_27.01.2025,22846531,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-01-27,2025-03-12,AND_Тонометры авто_27.01.2025,22846516,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
470,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
471,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
472,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
473,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [7]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Automatically get yesterday's date
yesterday = date.today().replace(day=date.today().day - 1)

specific_date = str(yesterday)  # Replace with your desired date

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [8]:
# Assume campaigns_guten DataFrame is already defined and contains 'advertId'
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, specific_date):
    all_data = []
    for idx, chunk in enumerate(chunks): 
        #Construct the payload for the current chunk
        payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_data(campaign_chunks, headers_guten, specific_date)
# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

Data retrieved successfully for chunk 1
Data retrieved successfully for chunk 2
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 4: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 5: 400, {"error":"Invalid Params: there are no companies with correct intervals"}


### Giper

In [9]:
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]
all_campaign_data_giper = []

def fetch_campaign_data(chunks, headers, specific_date):
    all_data = []
    for idx, chunk in enumerate(chunks):        
        payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]
        response = requests.post(url, headers=headers, json=payload)
        time.sleep(65)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

all_campaign_data_giper = fetch_campaign_data(campaign_chunks, headers_giper, specific_date)
campaign_giper_df = pd.json_normalize(all_campaign_data_giper)

Data retrieved successfully for chunk 1


### Kitchen

In [10]:
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

all_campaign_data_kitchen = []
def fetch_campaign_data(chunks, headers, specific_date):
    all_data = []
    for idx, chunk in enumerate(chunks):
        payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]
        response = requests.post(url, headers=headers, json=payload)
        time.sleep(65)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks, headers_kitchen, specific_date)

campaign_kitchen_df = pd.json_normalize(all_campaign_data_kitchen)

Data retrieved successfully for chunk 1


### Smart

In [11]:
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, specific_date):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_smart = fetch_campaign_data(campaign_chunks, headers_smart, specific_date)

Data retrieved successfully for chunk 1


In [12]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-03-13 03:00:00,252275489,Вытяжка кухонная 60 см встраиваемая телескопич...,368,11,2.99,3.20,35.16,1,0,0.0,0,0,19297685
1,2025-03-13 03:00:00,320765246,,16,0,0.00,0.00,1.49,0,0,0.0,0,0,19297685
2,2025-03-13 03:00:00,252275487,Вытяжка кухонная 60 см встраиваемая телескопич...,157,3,1.91,4.34,13.02,0,0,0.0,0,0,19297685
3,2025-03-13 03:00:00,320765243,,10,0,0.00,0.00,0.79,0,0,0.0,0,0,19297685
4,2025-03-13 03:00:00,286300575,Вытяжка кухонная 60 см подвесная ECO,17,0,0.00,0.00,1.38,0,0,0.0,0,0,19297685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2025-03-13 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,162,2,1.23,12.88,25.76,0,0,0.0,0,0,18870619
692,2025-03-13 03:00:00,46292575,Яйцеварка электрическая автоматическая E 9,350,12,3.43,5.69,68.23,0,0,0.0,0,0,18870619
693,2025-03-13 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,623,15,2.41,6.16,92.43,1,0,0.0,0,0,18870619
694,2025-03-13 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,324,21,6.48,2.34,49.07,0,0,0.0,0,0,18870619


In [16]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId","advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId","advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId","advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId","advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,23806943,2025-03-13 03:00:00,Вакууматор для продуктов с запайкой VC 11,454,8,1.623333,10.703333,69.09,0,0,0.000000,0,0,18870595,WB-GutenTech,Wildberries
1,25088481,2025-03-13 03:00:00,Эпилятор женский электрический Silk-epil 9-880,243,7,2.373333,4.886667,54.68,0,0,0.000000,0,0,21204285,WB-GutenTech,Wildberries
2,25623718,2025-03-13 03:00:00,Бритва мужская электрическая S3 300BT с триммером,108,6,8.630000,1.920000,9.48,0,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
3,25623718,2025-03-13 03:00:00,Бритва мужская электрическая S3 300BT с триммером,523,30,4.776667,1.960000,107.22,3,1,3.703333,1,6679,21202745,WB-GutenTech,Wildberries
4,25624340,2025-03-13 03:00:00,Электробритва Series 3 300s,117,2,2.296667,2.666667,9.73,0,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,235490802,2025-03-13 03:00:00,Эпилятор S9 SES 9-030,10,4,33.333333,0.016667,0.62,0,0,0.000000,0,0,20822563,WB-Smart-Market,Wildberries
450,235599972,2025-03-13 03:00:00,Электробритва Series 9 9510s,403,20,4.680000,1.140000,42.77,3,1,3.333333,1,22823,15562363,WB-Smart-Market,Wildberries
451,253299215,2025-03-13 03:00:00,Триммер для бороды и усов BT3421,211,14,5.550000,0.770000,20.35,4,0,0.000000,0,0,19597465,WB-Smart-Market,Wildberries
452,288465487,2025-03-13 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.000000,0.000000,0.06,0,0,0.000000,0,0,23236144,WB-Smart-Market,Wildberries


In [17]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,23806943,2025-03-13 03:00:00,Вакууматор для продуктов с запайкой VC 11,454,8,69.09,0,0,0,0,18870595,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Вакуумные упаковщики_2 SKU_25.07.24,Идут показы,Автоматическая кампания
1,25088481,2025-03-13 03:00:00,Эпилятор женский электрический Silk-epil 9-880,243,7,54.68,0,0,0,0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
2,25623718,2025-03-13 03:00:00,Бритва мужская электрическая S3 300BT с триммером,108,6,9.48,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
3,25623718,2025-03-13 03:00:00,Бритва мужская электрическая S3 300BT с триммером,523,30,107.22,3,1,1,6679,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
4,25624340,2025-03-13 03:00:00,Электробритва Series 3 300s,117,2,9.73,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,235490802,2025-03-13 03:00:00,Эпилятор S9 SES 9-030,10,4,0.62,0,0,0,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-03-05,Braun_Эпиляторы_22.10.2024,Кампания на паузе,Автоматическая кампания
450,235599972,2025-03-13 03:00:00,Электробритва Series 9 9510s,403,20,42.77,3,1,1,22823,15562363,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
451,253299215,2025-03-13 03:00:00,Триммер для бороды и усов BT3421,211,14,20.35,4,0,0,0,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания
452,288465487,2025-03-13 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.06,0,0,0,0,23236144,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания


## Статистика карточек товаров по дням
https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history

Response Schema: application/json

- data / Array of objects 
    - nmID / integer <int32> / Артикул W
    - imtName / string / Наименование карточки товар
    - vendorCode/ string/ Артикул продавц
    - history/ Array of objects / Array
        - dt/ string <date> / Дата
        - openCardCount / integer <int32> / Количество переходов в карточку товар
        - addToCartCount / integer <int32> / Положили в корзину, шту
        - ordersCount / integer <int32> / Заказали товаров, ш
        - ordersSumRub / integer <int32> / Заказали на сумму, руб/ buyoutsCount/ integer <int32/ Выкупили товаров, шт
        - buyoutsSumRub / integer <int32> / Выкупили на сумму, руб
        - buyoutPercent / integer <int32> / Процент выкупа, % (Какой процент посетителей, заказавших товар, его выкупили. Без учёта товаров, которые еще доставляются покупателю.)
        - addToCartConversion / numbe / Конверсия в корзину, % (Какой процент посетителей, открывших карточку товара, добавили товар в корзину)
        - cartToOrderConversion / integer <int32> / Конверсия в заказ, % (Какой процент посетителей, добавивших товар в корзину, сделали заказ)
- error/ boolean / Флаг ошибки
- errorText/ string / Описание ошибки
- additionalErrors / Array of objects / Дополнительные ошибки
        - field/ string / Структура, где допущена ошибки
        - description/ string / Описание

In [18]:
# API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history"

# Define headers and project names
projects = {
    'WB-GutenTech': headers_guten,
    'WB-ГиперМаркет': headers_giper,
    'WB-KitchenAid': headers_kitchen,
    'WB-Smart-Market': headers_smart
}

# Calculate yesterday's date
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# Define the period for the report (only yesterday)
period = {
    "begin": yesterday,
    "end": yesterday
}

# Function to send a single POST request
def send_request(nm_ids, period, headers):
    request_body = {
        "nmIDs": nm_ids,
        "period": period
    }
    response = requests.post(url, headers=headers, json=request_body)
    return response

# Function to fetch data in batches
def fetch_data_in_batches(nm_ids, period, headers):
    all_data = []
    batch_size = 20  # Maximum allowed nmIDs per request
    requests_per_minute = 3  # Maximum allowed requests per minute
    interval = 60 / requests_per_minute  # Time interval between requests in seconds

    for i in range(0, len(nm_ids), batch_size):
        batch = nm_ids[i:i + batch_size]
        response = send_request(batch, period, headers)

        if response.status_code == 200:
            try:
                data = response.json()
                if not data.get('error') and 'data' in data:
                    all_data.extend(data['data'])
                    print(f"Data retrieved successfully for batch {i // batch_size + 1}")
                else:
                    print(f"Error in response for batch {i // batch_size + 1}: {data.get('errorText', 'No error text')}")
            except ValueError as e:
                print(f"Failed to decode JSON for batch {i // batch_size + 1}: {e}")
        else:
            print(f"Error for batch {i // batch_size + 1}: {response.status_code}, {response.text}")

        time.sleep(interval)  # Respect the rate limit

    return all_data

# List to store DataFrames for each project
dataframes = []

# Process each project
for project_name, headers in projects.items():
    # Filter the DataFrame for the current project
    filtered_df = df_final[df_final['Project'] == project_name]
    unique_nmId_values = filtered_df['nmId'].unique().tolist()
    print(f"Total unique nmId values for {project_name}:", len(unique_nmId_values))  # Debugging line

    # Fetch data for all unique nmId values
    all_data = fetch_data_in_batches(unique_nmId_values, period, headers)

    # Flatten the nested 'history' data for easier analysis
    flattened_data = []
    for item in all_data:
        nmID = item['nmID']
        imtName = item['imtName']
        vendorCode = item['vendorCode']
        for history in item['history']:
            history_entry = {
                'nmID': nmID,
                'imtName': imtName,
                'vendorCode': vendorCode,
                'dt': history['dt'],
                'openCardCount': history['openCardCount'],
                'addToCartCount': history['addToCartCount'],
                'addToCartConversion': history['addToCartConversion'],
                'ordersCount': history['ordersCount'],
                'ordersSumRub': history['ordersSumRub'],
                'cartToOrderConversion': history['cartToOrderConversion'],
                'buyoutsCount': history['buyoutsCount'],
                'buyoutsSumRub': history['buyoutsSumRub'],
                'buyoutPercent': history['buyoutPercent']
            }
            flattened_data.append(history_entry)

    # Convert the flattened data to a DataFrame
    df_copy = pd.DataFrame(flattened_data)
    print(f"Data for {project_name}:\n", df_copy.head())

    # Add the DataFrame to the list
    dataframes.append(df_copy)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:\n", combined_df.head())
df_copy=combined_df.copy()

Total unique nmId values for WB-GutenTech: 200
Data retrieved successfully for batch 1
Data retrieved successfully for batch 2
Data retrieved successfully for batch 3
Data retrieved successfully for batch 4
Data retrieved successfully for batch 5
Data retrieved successfully for batch 6
Data retrieved successfully for batch 7
Data retrieved successfully for batch 8
Data retrieved successfully for batch 9
Data retrieved successfully for batch 10
Data for WB-GutenTech:
        nmID                                            imtName  \
0  25088481     Эпилятор женский электрический Silk-epil 9-880   
1  52469734  Вакууматор для продуктов с пакетами Vacu OneTouch   
2  26025671                   Электрическая вафельница WA 3606   
3  46292575         Яйцеварка электрическая автоматическая E 9   
4  26026221                   Электрическая вафельница WA 3492   

       vendorCode          dt  openCardCount  addToCartCount  \
0        Б0035166  2025-03-13             35               0   
1  

In [19]:
df_copy=combined_df.copy()
df_final_copy=df_final.copy()

In [20]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['day'] = pd.to_datetime(df_final_copy['day']).dt.date
df_copy['dt'] = pd.to_datetime(df_copy['dt']).dt.date

# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'dt': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df_2 = pd.merge(
    df_final_copy,
    df_copy[['nmId', 'day', 'ordersCount', 'ordersSumRub','addToCartCount']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df_2['ordersCount'].fillna(0, inplace=True)
merged_df_2['ordersSumRub'].fillna(0, inplace=True)
merged_df_2['addToCartCount'].fillna(0, inplace=True)


# Display the merged DataFrame
merged_df_2

C:\Users\User\AppData\Local\Temp\ipykernel_6996\76939134.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_2['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_6996\76939134.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,23806943,2025-03-13,Вакууматор для продуктов с запайкой VC 11,454,8,69.09,0,0,0,0,...,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Вакуумные упаковщики_2 SKU_25.07.24,Идут показы,Автоматическая кампания,0,0,2
1,25088481,2025-03-13,Эпилятор женский электрический Silk-epil 9-880,243,7,54.68,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,0
2,25623718,2025-03-13,Бритва мужская электрическая S3 300BT с триммером,108,6,9.48,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,1,7767,4
3,25623718,2025-03-13,Бритва мужская электрическая S3 300BT с триммером,523,30,107.22,3,1,1,6679,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,1,7767,4
4,25624340,2025-03-13,Электробритва Series 3 300s,117,2,9.73,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,1,7485,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,235490802,2025-03-13,Эпилятор S9 SES 9-030,10,4,0.62,0,0,0,0,...,Wildberries,2100-01-01,2024-10-22,2025-03-05,Braun_Эпиляторы_22.10.2024,Кампания на паузе,Автоматическая кампания,0,0,0
450,235599972,2025-03-13,Электробритва Series 9 9510s,403,20,42.77,3,1,1,22823,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,1,28890,3
451,253299215,2025-03-13,Триммер для бороды и усов BT3421,211,14,20.35,4,0,0,0,...,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,1,3443,5
452,288465487,2025-03-13,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.06,0,0,0,0,...,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания,0,0,0


In [21]:
merged_df_2

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,23806943,2025-03-13,Вакууматор для продуктов с запайкой VC 11,454,8,69.09,0,0,0,0,...,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Вакуумные упаковщики_2 SKU_25.07.24,Идут показы,Автоматическая кампания,0,0,2
1,25088481,2025-03-13,Эпилятор женский электрический Silk-epil 9-880,243,7,54.68,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,0
2,25623718,2025-03-13,Бритва мужская электрическая S3 300BT с триммером,108,6,9.48,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,1,7767,4
3,25623718,2025-03-13,Бритва мужская электрическая S3 300BT с триммером,523,30,107.22,3,1,1,6679,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,1,7767,4
4,25624340,2025-03-13,Электробритва Series 3 300s,117,2,9.73,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,1,7485,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,235490802,2025-03-13,Эпилятор S9 SES 9-030,10,4,0.62,0,0,0,0,...,Wildberries,2100-01-01,2024-10-22,2025-03-05,Braun_Эпиляторы_22.10.2024,Кампания на паузе,Автоматическая кампания,0,0,0
450,235599972,2025-03-13,Электробритва Series 9 9510s,403,20,42.77,3,1,1,22823,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,1,28890,3
451,253299215,2025-03-13,Триммер для бороды и усов BT3421,211,14,20.35,4,0,0,0,...,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,1,3443,5
452,288465487,2025-03-13,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.06,0,0,0,0,...,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания,0,0,0


## Insert the data

In [ ]:
password = os.getenv('ClickHouse')

In [ ]:
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [ ]:
# Ensure date columns are in the correct format for ClickHouse
merged_df_2['day'] = pd.to_datetime(merged_df_2['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merged_df_2.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'sum_price', 'advertId', 'Project', 'Marketplace','endTime', 'createTime', 'startTime', 
    'name_campaign', 'status', 'type','ordersCount', 'ordersSumRub','addToCartCount'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merged_df_2[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data inserted successfully!
